<a href="https://colab.research.google.com/github/dgutiluns/water-anomaly-ca/blob/main/cleaning_sample_and_streamcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from google.colab import drive

In [ ]:
drive.mount('/content/drive')


path = '/content/drive/MyDrive/Year 3 Berk/Data Discovery Group Folder/sample_with_streamcat.csv'
df = pd.read_csv(path)
df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-283428700.py:5: DtypeWarning: Columns (28,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,_id,Program,ParentProject,Project,StationName,StationCode,SampleDate,CollectionTime,LocationCode,CollectionDepth,...,prism_norm_8110.precip8110ws,prism_norm_8110.tmean8110ws,prism_norm_8110.tmax8110ws,prism_norm_8110.tmin8110ws,wetness.wetindexws,wetness.wetindexcat,runoff.runoffws,runoff.runoffcat,landfire_veg.pctnonagintrodmanagvegws,landfire_veg.pctnonagintrodmanagvegcat
0,1,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,0.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
1,2,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,3.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
2,3,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,6.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
3,4,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,9.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
4,5,Proposition 84,Prop84-City of San Diego,City of San Diego Hypolimnetic Oxygenation System,HGA,HGA,2017-04-18T00:00:00,00:00,Not Recorded,12.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8610,49978,Total Maximum Daily Load,Nutrient TMDL for Santa Margarita Watershed,Nutrient TMDL for Santa Margarita Watershed,Pipe discharging to Brow Ditch on Huffstatler St.,HST02,2022-01-10T00:00:00,10:12,Not Recorded,-88.0,...,433.220501,17.602009,24.055591,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69
8611,49984,Total Maximum Daily Load,Nutrient TMDL for Santa Margarita Watershed,Nutrient TMDL for Santa Margarita Watershed,Rainbow Creek @ Old Highway 395,RBC04,2022-01-10T00:00:00,10:26,Not Recorded,-88.0,...,433.220501,17.602009,24.055591,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69
8612,49987,Total Maximum Daily Load,Nutrient TMDL for Santa Margarita Watershed,Nutrient TMDL for Santa Margarita Watershed,Rainbow Creek @ Old Highway 395,RBC04,2022-01-10T00:00:00,10:26,Not Recorded,-88.0,...,433.220501,17.602009,24.055591,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69
8613,49993,Total Maximum Daily Load,Nutrient TMDL for Santa Margarita Watershed,Nutrient TMDL for Santa Margarita Watershed,Rainbow Glen Tributary to Rainbow Creek,RGT01,2022-01-10T00:00:00,10:55,Not Recorded,-88.0,...,426.989420,17.604599,23.618237,11.585818,617.7526,617.7526,27.0000,27.0000,2.69,2.69


In [ ]:
# Work on a deep copy so the original df stays intact
df_proc = df.copy(deep=True)

# 1) Coerce numeric on the copy
num_cols = ["Result","RL","MDL","DilutionFactor","Latitude","Longitude"]
for c in num_cols:
    if c in df_proc.columns:
        df_proc[c] = pd.to_numeric(df_proc[c], errors="coerce")

# 2) Fill censored values (Result -> RL -> MDL)
mask_empty  = df_proc["Result"].isna()
mask_rl_bad = df_proc["RL"].isna() | (df_proc["RL"] < 0)

df_proc["Result_filled"] = df_proc["Result"]
df_proc.loc[mask_empty & ~mask_rl_bad, "Result_filled"] = df_proc.loc[mask_empty & ~mask_rl_bad, "RL"]
df_proc.loc[mask_empty &  mask_rl_bad, "Result_filled"] = df_proc.loc[mask_empty &  mask_rl_bad, "MDL"]

# Audit flag
df_proc["SubbedFrom"] = np.where(mask_empty & ~mask_rl_bad, "RL",
                         np.where(mask_empty & mask_rl_bad, "MDL", "Result"))

# 3) Normalize units to µg/L
def to_ugL(val, unit):
    if pd.isna(val): return np.nan
    u = (str(unit) if unit is not None else "").strip().lower()
    if u in ["µg/l","ug/l","micrograms per liter","microgram per liter"]:
        return float(val)
    if u in ["mg/l","milligrams per liter","milligram per liter"]:
        return float(val) * 1000.0
    if u in ["ng/l","nanograms per liter","nanogram per liter"]:
        return float(val) * 0.001
    # Unknown → pass through
    return float(val)

df_proc["Result_ugL"] = [to_ugL(v, u) for v, u in zip(df_proc["Result_filled"], df_proc.get("Unit", np.nan))]


In [ ]:
# Overall missingness table
summary = (
    pd.DataFrame({
        "column": df_proc.columns,
        "dtype": [str(df_proc[c].dtype) for c in df_proc.columns],
        "n_missing": [df_proc[c].isna().sum() for c in df_proc.columns],
    })
    .assign(n=len(df_proc))
)
summary["missing_rate"] = summary["n_missing"] / summary["n"]

# Target substitution audit
summary_target = pd.DataFrame({
    "total_rows": [len(df_proc)],
    "result_non_null": [df_proc["Result"].notna().sum()],
    "subbed_from_RL": [int((df_proc["SubbedFrom"]=="RL").sum())],
    "subbed_from_MDL": [int((df_proc["SubbedFrom"]=="MDL").sum())],
    "units_missing_in_target": [int(df_proc.loc[df_proc["Result"].isna(), "Unit"].isna().sum())]
})
summary.sort_values("missing_rate", ascending=False, inplace=True)

print("Target substitution audit:")
display(summary_target)
print("\nTop 30 columns by missing rate:")
display(summary.head(30))



Target substitution audit:


,total_rows,result_non_null,subbed_from_RL,subbed_from_MDL,units_missing_in_target
0,8615,7901,410,304,0



Top 30 columns by missing rate:


,column,dtype,n_missing,n,missing_rate
21,Observation,float64,8615,8615,1.000000
35,GroupSamples,float64,8615,8615,1.000000
53,StartingBank,float64,8615,8615,1.000000
55,UnitDistanceFromBank,float64,8615,8615,1.000000
54,DistanceFromBank,float64,8615,8615,1.000000
57,UnitStreamWidth,float64,8615,8615,1.000000
52,OccupationMethod,float64,8615,8615,1.000000
60,HydroMod,float64,8615,8615,1.000000
59,UnitStationWaterDepth,float64,8615,8615,1.000000
58,StationWaterDepth,float64,8615,8615,1.000000


In [ ]:
# Columns that have any missingness
cols_with_missing = summary.loc[summary["n_missing"] > 0, "column"].tolist()

def miss_rate_by_group(df_like, group_col, cols):
    out = []
    for c in cols:
        mr = df_like.groupby(group_col)[c].apply(lambda s: s.isna().mean())
        out.append(mr.rename(c))
    return pd.concat(out, axis=1)

miss_analyte = miss_rate_by_group(df_proc, "Analyte", cols_with_missing)

# Worst offenders averaged across analytes
mean_miss_by_col = miss_analyte.mean(axis=0).sort_values(ascending=False)
print("Columns with highest average missing rate across analytes:")
display(mean_miss_by_col.head(20))

# Save wide table
# miss_analyte.to_csv("missingness_by_analyte.csv")
# print("Wrote missingness_by_analyte.csv")


Columns with highest average missing rate across analytes:


,0
Observation,1.000000
GroupSamples,1.000000
StartingBank,1.000000
UnitDistanceFromBank,1.000000
DistanceFromBank,1.000000
UnitStreamWidth,1.000000
OccupationMethod,1.000000
HydroMod,1.000000
UnitStationWaterDepth,1.000000
StationWaterDepth,1.000000


In [24]:
# ======================
# CLEAN TABLE for MERGE (no encoding, no matrix yet)
# ======================

# 1) Normalize non-detects (ND) & rebuild substituted numeric target
if "ResultQualCode" in df_proc.columns:
    nd_mask = df_proc["ResultQualCode"].astype(str).str.upper().eq("ND")
    df_proc.loc[nd_mask, "Result"] = np.nan  # blank Result for ND

mask_empty  = df_proc["Result"].isna()
mask_rl_ok  = df_proc["RL"].notna() & (df_proc["RL"] >= 0)

df_proc["Result_filled"] = df_proc["Result"]
df_proc.loc[mask_empty & mask_rl_ok,  "Result_filled"] = df_proc.loc[mask_empty & mask_rl_ok,  "RL"]
df_proc.loc[mask_empty & ~mask_rl_ok, "Result_filled"] = df_proc.loc[mask_empty & ~mask_rl_ok, "MDL"]
df_proc["SubbedFrom"] = np.where(~mask_empty, "Result", np.where(mask_rl_ok, "RL","MDL"))

# 2) Parse date + time -> timestamp and seasonal features
def _parse_dt(row):
    d = pd.to_datetime(row.get("SampleDate", pd.NaT), errors="coerce", utc=True)
    t = str(row.get("CollectionTime", "") or "").strip()
    if pd.notna(d) and ":" in t:
        try:
            hh, mm, *rest = t.split(":")
            ss = int(rest[0]) if rest else 0
            d = d.tz_convert(None).replace(hour=int(hh), minute=int(mm), second=ss)
        except Exception:
            pass
    return d

df_proc["SampleDateTime"] = df_proc.apply(_parse_dt, axis=1)
df_proc["Year"]  = df_proc["SampleDateTime"].dt.year
df_proc["Month"] = df_proc["SampleDateTime"].dt.month
df_proc["DOY"]   = df_proc["SampleDateTime"].dt.dayofyear
df_proc["Season"] = pd.Categorical(
    pd.cut(df_proc["Month"], bins=[0,3,6,9,12], labels=["Winter","Spring","Summer","Fall"], include_lowest=True)
)

# 3) Drop columns that are 100% missing in your snapshot
all_null_cols = [
    "Observation","GroupSamples","StartingBank","UnitDistanceFromBank","DistanceFromBank","UnitStreamWidth",
    "OccupationMethod","HydroMod","UnitStationWaterDepth","StationWaterDepth","DownStreamLength","UpstreamLength",
    "ChannelWidth","LocationDetailWQComments","HydroModLoc","TotalReach","LocationDetailBAComments"
]
df_proc.drop(columns=[c for c in all_null_cols if c in df_proc.columns], errors="ignore", inplace=True)

# 4) Keep ALL rows (you already filtered analytes upstream); make a working copy
df_mass = df_proc.copy()

# 5) Coordinate sanity + target columns (uses your existing Result_ugL from earlier cell)
if "Latitude" in df_mass:  df_mass.loc[~df_mass["Latitude"].between(-90, 90), "Latitude"] = np.nan
if "Longitude" in df_mass: df_mass.loc[~df_mass["Longitude"].between(-180, 180), "Longitude"] = np.nan

# Guard against negatives before log1p to avoid RuntimeWarnings
if "Result_ugL" in df_mass:
    df_mass.loc[df_mass["Result_ugL"] < 0, "Result_ugL"] = np.nan
    df_mass["Result_ugL_log"] = np.log1p(df_mass["Result_ugL"])
else:
    raise ValueError("Expected 'Result_ugL' from the previous unit-normalization cell.")

# Make a copy BEFORE dropping target-missing (useful for later triage/exports if you want)
df_mass_all = df_mass.copy()

# For modeling we’ll eventually need a target; for MERGE you can keep everything.
# Here, we'll keep rows with a defined numeric target (common choice), but if you want to
# keep all rows for merging, comment out the next line.
df_mass = df_mass[df_mass["Result_ugL_log"].notna()].copy()

# 6) Remove leakage / IDs / admin & long free-text (keep StationCode & SampleDateTime for merging)
# NOTE: we KEEP StationCode and SampleDateTime here because you said you'll merge on these.
drop_cols = [
    # pure IDs that won't help merge and can leak noise
    "_id","SampleID","LabSampleID","LabBatch","BatchVerification","ComplianceCode","SubmissionCode",
    # long comments
    "ResultsComments","BatchComments","SampleComments","CollectionComments",
    # raw target & audits & unit text (keep ResultQualCode though)
    "Result","Result_filled","Unit","SubbedFrom",
    # redundant raw date/time columns (we keep SampleDateTime instead)
    "SampleDate","CollectionTime",
    # other rarely-useful admin (you can add back if you plan to use in merge keys)
    "Program","ParentProject","Project","CalibrationDate","PrepPreservationDate",
    "DigestExtractDate","AnalysisDate","LabAgency","SubmittingAgency","StationName","LocationCode"
]
df_mass.drop(columns=[c for c in drop_cols if c in df_mass.columns], errors="ignore", inplace=True)

# 7) Promote numeric-like object columns (StreamCat etc.) to numeric if ≥90% parseable
for c in df_mass.select_dtypes(include=["object"]).columns.tolist():
    s = pd.to_numeric(df_mass[c], errors="coerce")
    if s.notna().mean() >= 0.90:
        df_mass[c] = s

# 7b) AUTO-PRUNE junky categoricals: near-constant or hyper-cardinal (keeps merge keys)
cat_cols = [c for c in df_mass.select_dtypes(include=["object","category"]).columns
            if c not in {"StationCode"}]  # never drop StationCode

near_constant = [c for c in cat_cols if df_mass[c].dropna().nunique() <= 1]

def too_cardinal(s, n_rows):
    u = s.dropna().nunique()
    return (u > 0.20 * n_rows) or (u > 200)

n_rows = len(df_mass)
hyper_card = [c for c in cat_cols if too_cardinal(df_mass[c], n_rows)]

to_drop_auto = sorted(set(near_constant + hyper_card))
df_mass.drop(columns=to_drop_auto, inplace=True, errors="ignore")

print("Auto-pruned near-constant cats:", near_constant)
print("Auto-pruned hyper-cardinal   :", [c for c in hyper_card if c not in near_constant])

# 8) (FINAL for merge) Select a tidy set of columns to hand to the weather-merge step
#    - keys to merge on (adjust to the keys used by your weather table)
#    - target columns
#    - cleaned predictors (numeric + curated small categoricals)
merge_key_cols = [c for c in ["StationCode","COMID","SampleDateTime","Latitude","Longitude","Year","Month","DOY","Season"] if c in df_mass.columns]

target_cols = [c for c in ["Result_ugL","Result_ugL_log","ResultQualCode"] if c in df_mass.columns]

# keep a few compact categoricals likely to matter (you can tweak this list)
keep_small_cats = [c for c in ["Analyte","MatrixName","MethodName","SampleTypeCode",
                               "CollectionMethodName","PositionWaterColumn","Datum","UnitCollectionDepth"]
                   if c in df_mass.columns]

# everything numeric that isn't obviously a key/target (StreamCat, soils, impervious, etc.)
numeric_predictors = [c for c in df_mass.select_dtypes(include=[np.number]).columns
                      if c not in {"Result_ugL","Result_ugL_log","Latitude","Longitude","Year","Month","DOY"}]  # Lat/Lon/Year/Month/DOY already in keys

df_clean_for_merge = df_mass[merge_key_cols + target_cols + keep_small_cats + numeric_predictors].copy()

print("df_clean_for_merge shape:", df_clean_for_merge.shape)
print("Columns kept (first 12):", df_clean_for_merge.columns[:12].tolist())


Auto-pruned near-constant cats: []
Auto-pruned hyper-cardinal   : []
df_clean_for_merge shape: (8611, 138)
Columns kept (first 12): ['StationCode', 'COMID', 'SampleDateTime', 'Latitude', 'Longitude', 'Year', 'Month', 'DOY', 'Season', 'Result_ugL', 'Result_ugL_log', 'ResultQualCode']


In [25]:
df_clean_for_merge

,StationCode,COMID,SampleDateTime,Latitude,Longitude,Year,Month,DOY,Season,Result_ugL,...,prism_norm_8110.precip8110ws,prism_norm_8110.tmean8110ws,prism_norm_8110.tmax8110ws,prism_norm_8110.tmin8110ws,wetness.wetindexws,wetness.wetindexcat,runoff.runoffws,runoff.runoffcat,landfire_veg.pctnonagintrodmanagvegws,landfire_veg.pctnonagintrodmanagvegcat
0,HGA,20329198.0,2017-04-18 00:00:00,33.04581,-117.12715,2017,4,108,Spring,50.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
1,HGA,20329198.0,2017-04-18 00:00:00,33.04581,-117.12715,2017,4,108,Spring,50.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
2,HGA,20329198.0,2017-04-18 00:00:00,33.04581,-117.12715,2017,4,108,Spring,492.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
3,HGA,20329198.0,2017-04-18 00:00:00,33.04581,-117.12715,2017,4,108,Spring,662.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
4,HGA,20329198.0,2017-04-18 00:00:00,33.04581,-117.12715,2017,4,108,Spring,759.0,...,477.242082,16.407631,24.216835,8.593340,687.8284,714.2622,38.9634,39.3049,16.61,1.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8610,HST02,22549103.0,2022-01-10 10:12:00,33.41174,-117.15196,2022,1,10,Winter,59.0,...,433.220501,17.602009,24.055591,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69
8611,RBC04,22549103.0,2022-01-10 10:26:00,33.41233,-117.15972,2022,1,10,Winter,1790000.0,...,433.220501,17.602009,24.055591,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69
8612,RBC04,22549103.0,2022-01-10 10:26:00,33.41233,-117.15972,2022,1,10,Winter,56.0,...,433.220501,17.602009,24.055591,11.144219,678.6893,688.0435,28.8988,28.6356,2.50,4.69
8613,RGT01,22548465.0,2022-01-10 10:55:00,33.41107,-117.18569,2022,1,10,Winter,1250000.0,...,426.989420,17.604599,23.618237,11.585818,617.7526,617.7526,27.0000,27.0000,2.69,2.69


In [27]:
output_path = '/content/drive/MyDrive/Year 3 Berk/Data Discovery Group Folder'

out_path = f"{output_path}/Data/clean_water_table_for_weather_merge.csv"
df_clean_for_merge.to_csv(out_path, index=False)
print("Saved to:", out_path)


Saved to: /content/drive/MyDrive/Year 3 Berk/Data Discovery Group Folder/clean_water_table_for_weather_merge.csv


In [ ]:
# @title
# Rows Dropped check:

# --- row-loss audit ---
tot = len(df_proc)

unit_str = df_proc.get("Unit", pd.Series(index=df_proc.index, dtype=object)).astype(str).str.lower().str.strip()
is_bio_unit = unit_str.str.contains(r"\b(cfu|mpn)\b", regex=True) | unit_str.str.contains(r"100\s*ml")
n_bio = int(is_bio_unit.sum())
n_mass = tot - n_bio

print(f"Total rows          : {tot}")
print(f"Bio (CFU/MPN) rows  : {n_bio}")
print(f"Mass/volume rows    : {n_mass}")

# Among mass rows, track target creation
m = df_proc.loc[~is_bio_unit].copy()
n_mass0 = len(m)
n_result_filled = int(m["Result_filled"].notna().sum())
n_ugl = int(m["Result_ugL"].notna().sum())

neg_target = int((m["Result_ugL"] < 0).sum())
print(f"Mass rows w/Result_filled notna : {n_result_filled} / {n_mass0}")
print(f"Mass rows w/Result_ugL notna    : {n_ugl} / {n_mass0}")
print(f"Negative Result_ugL (set NaN)   : {neg_target}")

# ND without RL/MDL (can be a big dropper)
if "ResultQualCode" in m:
    nd = m["ResultQualCode"].astype(str).str.upper().eq("ND")
    nd_no_sub = nd & m["Result"].isna() & m["RL"].isna() & m["MDL"].isna()
    print("ND rows with no RL & no MDL     :", int(nd_no_sub.sum()))


Total rows          : 8615
Bio (CFU/MPN) rows  : 4462
Mass/volume rows    : 4153
Mass rows w/Result_filled notna : 4153 / 4153
Mass rows w/Result_ugL notna    : 4153 / 4153
Negative Result_ugL (set NaN)   : 0
ND rows with no RL & no MDL     : 0


/tmp/ipython-input-805090087.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_bio_unit = unit_str.str.contains(r"\b(cfu|mpn)\b", regex=True) | unit_str.str.contains(r"100\s*ml")


In [ ]:
still_missing_target = df_proc["Result_ugL"].isna().values.sum(0)
still_missing_target

In [ ]:
# --- choose your target (optionally log-transform per analyte)
y = df_proc["Result_ugL"].to_numpy()
X = df_proc[feature_cols].to_numpy()

# 1) OOF predictions
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_pred = np.zeros_like(y, dtype=float)

for tr, te in kf.split(X):
    model = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=42)
    model.fit(X[tr], y[tr])
    oof_pred[te] = model.predict(X[te])

resid = y - oof_pred          # or np.log1p(y) - np.log1p(oof_pred)
abs_resid = np.abs(resid)

# 2) Isolation Forest on residuals
iso = IsolationForest(contamination=0.05, random_state=42)
labels = iso.fit_predict(abs_resid.reshape(-1, 1))   # -1 = outlier, 1 = inlier

df["residual"] = resid
df["if_label"] = labels
df["is_outlier"] = (labels == -1)


In [ ]:
context_features = ['Temperature', 'Rainfall', 'UnitCollectionDepth']  # add any other numeric features you have

# Contamination rate for Isolation Forest (expected fraction of anomalies)
contamination_rate = 0.01

# Small constant for log-transform
log_eps = 1e-6

# FUNCTION: detect anomalies per analyte
def detect_per_analyte(df, analyte_col='Analyte', result_col='Result'):
    results = []
    analytes = df[analyte_col].unique()

    for analyte in analytes:
        df_a = df[df[analyte_col] == analyte].copy()

        # Log-transform result if skewed
        if df_a[result_col].skew() > 1:
            df_a['Result_log'] = np.log(df_a[result_col] + log_eps)
            target_col = 'Result_log'
        else:
            df_a['Result_log'] = df_a[result_col]
            target_col = 'Result_log'

        # Select numeric features for modeling
        numeric_cols = [target_col] + [f for f in context_features if f in df_a.columns]
        X = df_a[numeric_cols].copy()

        # Standardize
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Fit Isolation Forest
        iso = IsolationForest(contamination=contamination_rate, random_state=42)
        iso.fit(X_scaled)
        df_a['anomaly_score'] = iso.decision_function(X_scaled)
        df_a['anomaly_label'] = iso.predict(X_scaled)  # -1 = outlier, 1 = normal

        # PCA for visualization
        if X_scaled.shape[1] > 1:  # more than just the analyte
            pca = PCA(n_components=2)
            pcs = pca.fit_transform(X_scaled)
            df_a['PC1'] = pcs[:, 0]
            df_a['PC2'] = pcs[:, 1]

        results.append(df_a)

    df_results = pd.concat(results, axis=0)
    return df_results

In [ ]:
# RUN PIPELINE
df_results = detect_per_analyte(df, analyte_col='Analyte', result_col='Result')

# Example: visualize anomalies for a specific analyte
analyte_to_plot = 'Nitrate'
df_plot = df_results[df_results['Analyte'] == analyte_to_plot]

plt.figure(figsize=(8,6))
plt.scatter(df_plot['PC1'], df_plot['PC2'], c=df_plot['anomaly_label'], cmap='coolwarm', alpha=0.6)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title(f'{analyte_to_plot} Anomalies (Isolation Forest + PCA)')
plt.show()

# df_results now has:
# - anomaly_score
# - anomaly_label (-1 = outlier)
# - optionally PC1/PC2 for visualization
# for each analyte measurement row